# Bermudan Swaption Pricing

Value a European Swaption using the Hull-White, Black-Derman-Toy and Black-Karasinski models. I analyse relationships to test the model pricing.

NOTE SENSITIVE TO EVEN-ODD NUMBER OF TIME STEPS ON TREE - INVESTIGATE

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *
from financepy.models.hw_tree import HWTree
from financepy.models.bk_tree import BKTree
from financepy.models.bdt_tree import BDTTree

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  14 Jul 2021 at 16:00 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Discount Curve

In [3]:
valuation_date = Date(1, 1, 2011)
settlement_date = valuation_date

Set up flat discount curve

In [4]:
rate = 0.0625
libor_curve = DiscountCurveFlat(valuation_date, 
                                  rate, 
                                  FrequencyTypes.SEMI_ANNUAL, 
                                  DayCountTypes.ACT_365F)

# The Underlying Swap

We begin with the underlying swap.

In [5]:
exercise_date = settlement_date.add_years(1)
swapMaturityDate = settlement_date.add_years(4)
swapFixedCoupon = 0.060
swapFixedFrequencyType = FrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = DayCountTypes.ACT_365F
swapType = SwapTypes.PAY 

We can value the forward starting swap.

In [6]:
swap = IborSwap(exercise_date,
                    swapMaturityDate,
                    swapType,
                    swapFixedCoupon,
                    swapFixedFrequencyType,
                    swapFixedDayCountType)

We can approximate the value using 2.5 as the approximate duration of a 3 year swap

In [7]:
(rate - swapFixedCoupon) * 2.5 * ONE_MILLION

6250.0000000000055

In [8]:
swapValue = swap.value(valuation_date, libor_curve, libor_curve)
print("Swap Value: %9.5f"% swapValue)

Swap Value: 6348.48951


The estimate was not bad !

In [9]:
swap.print_fixed_leg_pv()

START DATE: 01-JAN-2012
MATURITY DATE: 01-JAN-2015
COUPON (%): 6.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_365F
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    RATE      PAYMENT       DF          PV        CUM PV
02-JUL-2012  01-JAN-2012  02-JUL-2012   183  0.501370   6.00000     30082.19  0.91174104     27427.17     27427.17
01-JAN-2013  02-JUL-2012  01-JAN-2013   183  0.501370   6.00000     30082.19  0.88403799     26593.80     54020.97
01-JUL-2013  01-JAN-2013  01-JUL-2013   181  0.495890   6.00000     29753.42  0.85746580     25512.54     79533.51
01-JAN-2014  01-JUL-2013  01-JAN-2014   184  0.504110   6.00000     30246.58  0.83127172     25143.12    104676.64
01-JUL-2014  01-JAN-2014  01-JUL-2014   181  0.495890   6.00000     29753.42  0.80628556     23989.76    128666.39
01-JAN-2015  01-JUL-2014  01-JAN-2015   184  0.504110   6.00000     30246.58  0.78165495     23642.39    152308.78


In [10]:
swap.print_float_leg_pv()

START DATE: 01-JAN-2012
MATURITY DATE: 01-JAN-2015
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    IBOR      PAYMENT       DF          PV        CUM PV
02-APR-2012  01-JAN-2012  02-APR-2012    91  0.252778    6.18457     15633.23  0.92583837     14473.84     14473.84
02-JUL-2012  02-APR-2012  02-JUL-2012    90  0.250000    6.18480     15461.99  0.91174104     14097.34     28571.18
01-OCT-2012  02-JUL-2012  01-OCT-2012    89  0.247222    6.25429     15461.99  0.89785835     13882.68     42453.86
01-JAN-2013  01-OCT-2012  01-JAN-2013    90  0.250000    6.25329     15633.23  0.88403799     13820.37     56274.23
01-APR-2013  01-JAN-2013  01-APR-2013    90  0.250000    6.11632     15290.79  0.87072393     13314.06     69588.28
01-JUL-2013  01-APR-2013  01-JUL-2013    90  0.250000    6.18480     15461.99  0.85746580     13258.13     82846.42
01-OCT-2013  01-JUL-2013  01-OCT-2013    90  0.2500

# IborSwaption

This is a class for European-style swaptions. It implements a number of model implementations.

In [11]:
europeanSwaptionPay = IborSwaption(settlement_date,
                                      exercise_date,
                                      swapMaturityDate,
                                      SwapTypes.PAY,
                                      swapFixedCoupon,
                                      swapFixedFrequencyType,
                                      swapFixedDayCountType)

In [12]:
europeanSwaptionRec = IborSwaption(settlement_date,
                                      exercise_date,
                                      swapMaturityDate,
                                      SwapTypes.RECEIVE,
                                      swapFixedCoupon,
                                      swapFixedFrequencyType,
                                      swapFixedDayCountType)

In [13]:
europeanSwaptionPay

OBJECT TYPE: IborSwaption
SETTLEMENT DATE: 01-JAN-2011
EXERCISE DATE: 01-JAN-2012
SWAP FIXED LEG TYPE: SwapTypes.PAY
SWAP MATURITY DATE: 01-JAN-2015
SWAP NOTIONAL: 1000000
FIXED COUPON: 6.0
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.ACT_365F
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.THIRTY_E_360

Set the volatility to zero so that payer is pure intrinsic

### Black's Model

In [14]:
sigma = 0.00001

In [15]:
model = Black(sigma)

In [16]:
europeanSwaptionPay.value(valuation_date, libor_curve, model)

6348.489508767811

Let's check for put-call parity

In [17]:
sigma = 0.20

In [18]:
model = Black(sigma)

In [19]:
valuePay = europeanSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 15815.70649
REC Value: 9467.21698
PAY - REC: 6348.48951


Yes put-call parity is respected.

### BK Model

In [20]:
a = 0.01
sigma = 0.0000001
num_time_steps = 100
model = BKTree(sigma, a, num_time_steps)

In [21]:
valuePay = europeanSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6346.08093
REC Value:   0.00000
PAY - REC: 6346.08093


In [22]:
a = 0.01
sigma = 0.20
num_time_steps = 100
model = BKTree(sigma, a, num_time_steps)

In [23]:
valuePay = europeanSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 15746.57475
REC Value: 9400.49421
PAY - REC: 6346.08054


### BDT Model

In [24]:
sigma = 0.0000001
num_time_steps = 200
model = BDTTree(sigma, num_time_steps)

In [25]:
valuePay = europeanSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6347.28920
REC Value:   0.00000
PAY - REC: 6347.28920


In [26]:
sigma = 0.20
model = BDTTree(sigma, num_time_steps)

In [27]:
valuePay = europeanSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 15990.91739
REC Value: 9639.93743
PAY - REC: 6350.97996


### HW Model

In [28]:
sigma = 0.0000001
num_time_steps = 200
a = 0.01
model = HWTree(sigma, a, num_time_steps)

In [29]:
valuePay = europeanSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6348.45763
REC Value:   0.00000
PAY - REC: 6348.45763


In [30]:
sigma = 0.20 * rate
num_time_steps = 200
a = 0.01
model = HWTree(sigma, a, num_time_steps)

In [31]:
valuePay = europeanSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 16217.69298
REC Value: 9869.43817
PAY - REC: 6348.25481


# IborBermudanSwaption

I now examine FinBermudanSwaption and use the tree models to value it. But first let me price it on the tree models as a European option.

## Start By Only Allowing European Exercise

In [32]:
europeanBermSwaptionPay = IborBermudanSwaption(settlement_date,
                                                  exercise_date,
                                                  swapMaturityDate,
                                                  SwapTypes.PAY,
                                                  FinExerciseTypes.EUROPEAN,
                                                  swapFixedCoupon,
                                                  swapFixedFrequencyType,
                                                  swapFixedDayCountType)

In [33]:
europeanBermSwaptionRec = IborBermudanSwaption(settlement_date,
                                                  exercise_date,
                                                  swapMaturityDate,
                                                  SwapTypes.RECEIVE,
                                                  FinExerciseTypes.EUROPEAN,
                                                  swapFixedCoupon,
                                                  swapFixedFrequencyType,
                                                  swapFixedDayCountType)

### Black Karasinski Model

Setting the volatility close to zero so we only have intrinsic value.

In [34]:
blackVol = 0.0000001

In [35]:
sigma = blackVol
a = 0.01
num_time_steps = 200
model = BKTree(sigma, a, num_time_steps)

In [36]:
valuePay = europeanBermSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanBermSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6313.74545
REC Value:   0.00000
PAY - REC: 6313.74545


In [38]:
europeanBermSwaptionPay.print_swaption_value()

SWAP PV01: 2.538479623216061
CPN TIME:  1.0 FLOW 0.0
CPN TIME:  1.5013698630136987 FLOW 0.030246575342465755
CPN TIME:  2.0027397260273974 FLOW 0.03008219178082192
CPN TIME:  2.4986301369863013 FLOW 0.03008219178082192
CPN TIME:  3.0027397260273974 FLOW 0.029753424657534246
CPN TIME:  3.4986301369863013 FLOW 0.030246575342465755
CPN TIME:  4.002739726027397 FLOW 0.029753424657534246
CALL TIME:  1.0
CALL TIME:  1.5013698630136987
CALL TIME:  2.0027397260273974
CALL TIME:  2.4986301369863013
CALL TIME:  3.0027397260273974
CALL TIME:  3.4986301369863013
CALL TIME:  4.002739726027397


### Black Derman Toy Model

In [39]:
sigma = 0.0000001
num_time_steps = 100
model = BDTTree(sigma, num_time_steps)

In [40]:
valuePay = europeanBermSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanBermSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6311.83846
REC Value:   0.00000
PAY - REC: 6311.83846


In [41]:
sigma = 0.20
num_time_steps = 100
model = BDTTree(sigma, num_time_steps)

In [42]:
valuePay = europeanBermSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanBermSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 15937.51370
REC Value: 9618.31192
PAY - REC: 6319.20178


### Hull White Model

In [43]:
sigma = 0.0000001
a = 0.01
num_time_steps = 101
model = HWTree(sigma, a, num_time_steps)

In [44]:
valuePay = europeanBermSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanBermSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6314.50332
REC Value:   0.00000
PAY - REC: 6314.50332


In [45]:
sigma = 0.20 * rate
a = 0.01
num_time_steps = 101
model = HWTree(sigma, a, num_time_steps)

In [46]:
valuePay = europeanBermSwaptionPay.value(valuation_date, libor_curve, model)
valueRec = europeanBermSwaptionRec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 16237.06135
REC Value: 9922.55803
PAY - REC: 6314.50332


# Now allow Multiple Exercise Dates - Bermudan

Now allow exercise on ALL coupon dates PLUS expiry.

In [47]:
bermudan_swaption_pay = IborBermudanSwaption(settlement_date,
                                               exercise_date,
                                               swapMaturityDate,
                                               SwapTypes.PAY,
                                               FinExerciseTypes.BERMUDAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

In [48]:
bermudan_swaption_rec = IborBermudanSwaption(settlement_date,
                                               exercise_date,
                                               swapMaturityDate,
                                               SwapTypes.RECEIVE,
                                               FinExerciseTypes.BERMUDAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

### Black Karasinski Model

In [49]:
sigma = 0.20
a = 0.01
num_time_steps = 100
model = BKTree(sigma, a, num_time_steps)

In [50]:
valuePay = bermudan_swaption_pay.value(valuation_date, libor_curve, model)
valueRec = bermudan_swaption_rec.value(valuation_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 19235.85760
REC Value: 13043.39490
PAY - REC: 6192.46270


### Black-Derman-Toy Model

In [51]:
sigma = 0.20
num_time_steps = 100
model = BDTTree(sigma, num_time_steps)

In [52]:
valuePay = bermudan_swaption_pay.value(settlement_date, libor_curve, model)
valueRec = bermudan_swaption_rec.value(settlement_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 19500.06870
REC Value: 13318.31315
PAY - REC: 6181.75555


### Hull-White Model

In [53]:
sigma = 0.01
a = 0.01
num_time_steps = 100
model = HWTree(sigma, a)

In [54]:
valuePay = bermudan_swaption_pay.value(settlement_date, libor_curve, model)
valueRec = bermudan_swaption_rec.value(settlement_date, libor_curve, model)
valuePayRec = valuePay - valueRec
print("PAY Value: %9.5f"% valuePay)
print("REC Value: %9.5f"% valueRec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 16578.58987
REC Value: 10406.29704
PAY - REC: 6172.29283


The swaption is Bermudan so it can only be exercised on coupon payment dates

Copyright (c) 2020 Dominic O'Kane